In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, load_breast_cancer
from sklearn.preprocessing import StandardScaler

In [2]:

data = load_breast_cancer()


In [4]:
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X = StandardScaler().fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [5]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_neurons, output_dim):
        super().__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_neurons))
        layers.append(nn.ReLU())

        # Add hidden layers
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(hidden_neurons, hidden_neurons))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_neurons, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [6]:
def train_model(model, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(50):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    # Evaluate accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            pred = model(xb)
            predicted = torch.argmax(pred, dim=1)
            total += yb.size(0)
            correct += (predicted == yb).sum().item()
    accuracy = correct / total
    return accuracy


In [7]:
for layers in [1, 3, 5, 10, 20]:
    model = SimpleNN(input_dim=2, hidden_layers=layers, hidden_neurons=10, output_dim=2)
    acc = train_model(model, train_loader, test_loader)
    print(f"Layers: {layers}, Test Accuracy: {acc:.4f}")


Layers: 1, Test Accuracy: 0.9600
Layers: 3, Test Accuracy: 0.9500
Layers: 5, Test Accuracy: 0.9500
Layers: 10, Test Accuracy: 0.9450
Layers: 20, Test Accuracy: 0.5250


In [16]:
for neurons in [5, 10, 20, 50, 100]:
    model = SimpleNN(input_dim=2, hidden_layers=3, hidden_neurons=neurons, output_dim=2)
    acc = train_model(model, train_loader, test_loader)
    print(f"Neurons per layer: {neurons}, Test Accuracy: {acc:.4f}")


Neurons per layer: 5, Test Accuracy: 0.9750
Neurons per layer: 10, Test Accuracy: 0.9800
Neurons per layer: 20, Test Accuracy: 0.9650
Neurons per layer: 50, Test Accuracy: 0.9650
Neurons per layer: 100, Test Accuracy: 0.9650


In [17]:
data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [1]:
model = SimpleNN(input_dim=X.shape[1], hidden_layers=3, hidden_neurons=32, output_dim=2)


NameError: name 'SimpleNN' is not defined

In [30]:
accuracy = train_model(model, train_loader, test_loader, epochs=100)
print(f"Breast Cancer Test Accuracy: {accuracy:.4f}")


Breast Cancer Test Accuracy: 0.9561
